Copyright (c) 2021-2024, NVIDIA CORPORATION & AFFILIATES

SPDX-License-Identifier: BSD-3-Clause

In this notebook, we use a simple example to show how to use `cuquantum.CircuitToEinsum` 
to take a `cirq.Circuit` instance and run basic quantum circuit simulation using cuTensorNet.

In [1]:
import itertools

import cirq
from cirq.testing import random_circuit
import cupy as cp
import numpy as np

from cuquantum import contract, CircuitToEinsum

## Generate a random quantum circuit

The generated `circuit` object is of type `cirq.Circuit`.

In [2]:
num_qubits = 7
n_moments = 6
op_density = 0.9
gate_domain = {cirq.H: 1, 
               cirq.S: 1, 
               cirq.T: 1, 
               cirq.CNOT: 2, 
               cirq.CZ: 2}

circuit = random_circuit(num_qubits, n_moments, op_density=op_density, gate_domain=gate_domain, random_state=6)
print(circuit)

      ┌──┐       ┌──┐   ┌──┐   ┌───┐   ┌──┐
0: ────H─────@────T──────H────────X─────X─────
             │                    │     │
1: ────@─────X─────@─────────────@┼─────┼S────
       │           │             ││     │
2: ────┼H────H────H┼─────@──────T┼┼─────┼─────
       │           │     │       ││     │
3: ────@──────────S┼─────┼H──────┼┼─────┼─────
                   │     │       ││     │
4: ────────────────X─────┼───────X┼─────@─────
                         │        │
5: ────T──────────S──────┼────────@─────X─────
                         │              │
6: ────T─────H───────────@──────T───────@─────
      └──┘       └──┘   └──┘   └───┘   └──┘


## Construct the converter object targeting double precision

In this example, we generate the tensor operands as CuPy arrays (by setting `backend=cupy`).

In [3]:
myconverter = CircuitToEinsum(circuit, dtype='complex128', backend=cp)

### compute state vector $|\psi\rangle$

In [4]:
expression, operands = myconverter.state_vector()
sv = contract(expression, *operands)
print(f'wavefunction coefficient shape: {sv.shape}')

# check if the computed statevector is correct
simulator = cirq.Simulator(dtype=np.complex128)
result = simulator.simulate(circuit)
sv_cirq = result.state_vector().reshape((2,)*num_qubits)
print(f"is sv in agreement?", cp.allclose(sv, sv_cirq))

wavefunction coefficient shape: (2, 2, 2, 2, 2, 2, 2)
is sv in agreement? True


### calculate bitstring amplitude $\langle b| \psi\rangle$

In [5]:
bitstring = '0000000'

expression, operands = myconverter.amplitude(bitstring)
print(f'einsum expression:\n{expression}\n')

amplitude = contract(expression, *operands)
probability = abs(amplitude) ** 2
print(f'for bitstring {bitstring}, amplitude: {amplitude}, probability: {probability}\n')

amplitude_from_sv = sv[0,0,0,0,0,0,0]
amp_diff = abs(amplitude-amplitude_from_sv)
print(f'difference from state vector: {amp_diff}')

einsum expression:
a,b,c,d,e,f,g,ha,ijdb,kg,lf,mc,nm,ok,pqhj,ri,sp,tl,uvqe,wn,xs,yzwo,Ar,By,CDuv,EFtx,Gz,HIDF,JC,KLGE,I,J,B,A,H,L,K->

for bitstring 0000000, amplitude: (0.17677669529663714+0j), probability: 0.03125000000000009

difference from state vector: 2.7755575615628914e-17


### calculate batch of bistring amplitudes

In this example, we calculate a batch of bistring amplitudes $\langle 00000ij|\psi\rangle$ where the first 5 qubits are fixed at state $00000$ and the last two qubit states are batched. This is equivalent to computing a slice of the full state vector.

In [6]:
fixed_states = '00000'
fixed_index = tuple(map(int, fixed_states))
num_fixed = len(fixed_states)

# mapping of the first 5 qubits to the fixed state
fixed = dict(zip(myconverter.qubits[:num_fixed], fixed_states))

expression, operands = myconverter.batched_amplitudes(fixed)
batched_amplitudes = contract(expression, *operands)

for ibit, jbit in itertools.product(range(2), repeat=2):
    bitstring = fixed_states + str(ibit) + str(jbit)
    index = fixed_index + (ibit, jbit)
    amplitude = batched_amplitudes[(ibit, jbit)]
    amplitude_from_sv = sv[index]
    amp_diff = abs(amplitude-amplitude_from_sv)
    print(f'for bitstring {bitstring}, amplitude: {amplitude:.4f}, difference from state vector: {amp_diff:.4f}')

for bitstring 0000000, amplitude: 0.1768+0.0000j, difference from state vector: 0.0000
for bitstring 0000001, amplitude: 0.0000+0.0000j, difference from state vector: 0.0000
for bitstring 0000010, amplitude: 0.0000+0.0000j, difference from state vector: 0.0000
for bitstring 0000011, amplitude: 0.1250+0.1250j, difference from state vector: 0.0000


### compute expectation value $\langle \psi|\hat{O}| \psi\rangle$

In this example, we compute the expectation value for a pauli string $IIYXIXY$. For comparision, we compute the same value via contracting reduced density matrix with the operator.

In [7]:
pauli_string = 'IIYXIXY'
expression, operands = myconverter.expectation(pauli_string, lightcone=True)
expec = contract(expression, *operands)
print(f'expectation value for {pauli_string}: {expec}')

# expectation value from reduced density matrix
qubits = myconverter.qubits
where = qubits[2:4] + qubits[5:]
rdm_expression, rdm_operands = myconverter.reduced_density_matrix(where, lightcone=True)
rdm = contract(rdm_expression, *rdm_operands)

pauli_x = cp.asarray([[0,1],[1,0]], dtype=myconverter.dtype)
pauli_y = cp.asarray([[0,-1j], [1j,0]], dtype=myconverter.dtype)
pauli_z = cp.asarray([[1,0],[0,-1]], dtype=myconverter.dtype)
expec_from_rdm = cp.einsum('abcdABCD,aA,bB,cC,dD->', rdm, pauli_y, pauli_x, pauli_x, pauli_y)

print(f"is expectation value in agreement?", cp.allclose(expec, expec_from_rdm))

expectation value for IIYXIXY: (0.5000000000000016+0j)
is expectation value in agreement? True


### compute reduced density matrix $\rho\left( {\bar{p}|b_{\bar{f}}} \right) = Tr_{\bar{p}}\langle b_{\bar{f}}|\psi_{\bar{f}\cup\bar{p}}\rangle \langle \psi_{\bar{f}\cup\bar{p}}\rangle| b_{\bar{f}} \rangle$

In this example, we construct the reduced density matrix of qubits 0 & 1 (so the returned RDM should have $2^2\times2^2=16$ elements), with qubits 3 & 4 fixed at state 0

In [8]:
where = qubits[:2]
fixed = {qubits[3]: '0',
         qubits[4]: '0'}

# we set lightcone=True to reduce the size of the tensor network
expression, operands = myconverter.reduced_density_matrix(where, fixed=fixed, lightcone=True)
rdm = contract(expression, *operands)
print(f'shape of reduced density matrix: {rdm.shape}')

# check if the computed RDM is correct
rdm_from_sv = contract('abefg, ABefg->abAB', sv[:, :, :, 0, 0], sv[:, :, :, 0, 0].conj())
print(f"is rdm in agreement?", cp.allclose(rdm, rdm_from_sv))


shape of reduced density matrix: (2, 2, 2, 2)
is rdm in agreement? True
